In [1]:
import os
mingw_path = 'C:\\Program Files\\mingw-w64\\x86_64-6.3.0-posix-seh-rt_v5-rev1\\mingw64\\bin'
os.environ['PATH'] = mingw_path + ';' + os.environ['PATH']

In [2]:
% pylab inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
from xgboost.sklearn import XGBRegressor
from sklearn import cross_validation,metrics
from sklearn.grid_search import GridSearchCV

Populating the interactive namespace from numpy and matplotlib


C:\Users\Ambar Gupta\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\Users\Ambar Gupta\Anaconda3\lib\site-packages\sklearn\grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [3]:
train = pd.read_csv('train.csv')

In [4]:
train.head()

,TID,vendor_id,new_user,tolls_amount,tip_amount,mta_tax,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,rate_code,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,surcharge,fare_amount
0,AIX000345001,DST000401,NO,0.0,1.4,0.5,2015-04-20 04:18:25,2015-04-20 04:24:20,1,-74.003939,40.742894,1,N,-73.993369,40.734247,CRD,0.5,8.4
1,AIX000345002,DST000401,NO,0.0,1.0,0.5,2015-04-19 18:16:07,2015-04-19 18:22:06,3,-73.973864,40.752194,1,N,-73.958701,40.772533,CRD,0.0,8.5
2,AIX000345003,DST000401,NO,0.0,0.0,0.5,2015-04-06 08:04:32,2015-04-06 08:10:05,2,-73.954406,40.764420,1,N,-73.970780,40.758350,CSH,0.0,7.0
3,AIX000345004,DST000532,NO,0.0,1.8,0.5,2015-04-10 09:48:00,2015-04-10 10:00:00,2,-73.962345,40.767215,1,NaN,-73.975512,40.756867,CRD,0.0,11.3
4,AIX000345005,DST000401,NO,0.0,0.0,0.5,2015-04-15 13:12:48,2015-04-15 13:26:30,1,-74.004657,40.707434,1,N,-73.999369,40.721517,CSH,0.0,10.0


In [5]:
train.describe()

C:\Users\Ambar Gupta\Anaconda3\lib\site-packages\numpy\lib\function_base.py:3834: RuntimeWarning: Invalid value encountered in percentile
  RuntimeWarning)


,tolls_amount,tip_amount,mta_tax,passenger_count,pickup_longitude,pickup_latitude,rate_code,dropoff_longitude,dropoff_latitude,surcharge,fare_amount
count,1.658099e+06,1.492903e+06,1.658099e+06,1.658099e+06,1.608290e+06,1.624998e+06,1.658099e+06,1.653112e+06,1.649937e+06,1.558926e+06,1.658099e+06
mean,2.770844e-01,1.554779e+00,4.980460e-01,1.722504e+00,-7.258480e+01,3.998628e+01,1.035918e+00,-7.258127e+01,3.998414e+01,3.177296e-01,1.531551e+01
std,1.890095e+00,2.356677e+00,3.194602e-02,1.384761e+00,1.004539e+01,5.528833e+00,4.683282e-01,1.005460e+01,5.538230e+00,3.623541e-01,1.272185e+01
min,-1.175000e+01,0.000000e+00,-5.000000e-01,0.000000e+00,-9.908311e+01,-1.355188e+01,0.000000e+00,-1.190126e+02,-1.355188e+01,-1.000000e+00,0.000000e+00
25%,0.000000e+00,NaN,5.000000e-01,1.000000e+00,NaN,NaN,1.000000e+00,NaN,NaN,NaN,8.300000e+00
50%,0.000000e+00,NaN,5.000000e-01,1.000000e+00,NaN,NaN,1.000000e+00,NaN,NaN,NaN,1.150000e+01
75%,0.000000e+00,NaN,5.000000e-01,2.000000e+00,NaN,NaN,1.000000e+00,NaN,NaN,NaN,1.700000e+01
max,1.450090e+03,2.312000e+02,5.000000e-01,9.000000e+00,7.945679e+01,6.798689e+01,2.100000e+02,0.000000e+00,6.343189e+01,2.050000e+01,1.502890e+03


In [6]:
def convert_datetime(X):
    X['pickup_datetime'] = X['pickup_datetime'].apply(lambda x:datetime.strptime(x,'%Y-%m-%d %H:%M:%S'))
    X['dropoff_datetime'] = X['dropoff_datetime'].apply(lambda x:datetime.strptime(x,'%Y-%m-%d %H:%M:%S'))
    return X

In [7]:
def cal(X):
    df = pd.DataFrame()
    df['diff'] = X['dropoff_datetime'] - X['pickup_datetime']
    df['diff'] = df['diff'].apply(lambda x:divmod(x.days * 86400 + x.seconds, 60))
    df['diff'] = df['diff'].apply(lambda x:abs(x[0]+(x[1]/60)))
    return df['diff']

In [8]:
from math import cos, asin, sqrt
def distance(lat1, lon1, lat2, lon2):
    if(np.isnan(lat1) or np.isnan(lon1) or np.isnan(lat2) or np.isnan(lon2)):
        return 0
    else:
        p = 0.017453292519943295     #Pi/180
        a = 0.5 - cos((lat2 - lat1) * p)/2 + cos(lat1 * p) * cos(lat2 * p) * (1 - cos((lon2 - lon1) * p)) / 2
        return 12742 * asin(sqrt(a))

In [9]:
def convert_vendorid(s):
    if '401' in s:
        return 0
    else:
        return int(s[-1])

In [35]:
def update_test(test):
    print('updating datetime')
    test = convert_datetime(test)
    test['total_jrny'] = cal(test)
    print('done')
    print('updating vendorid')
    test['vendor_id'] = test['vendor_id'].apply(lambda x:convert_vendorid(x))
    print('done')      
    print('updating new_user')
    test['new_user'] = test['new_user'].fillna('NO')
    test['new_user'] = pd.Series(np.where(test.new_user.values=='yes',1,0),test.index)
    print('done')      
    print('updating tip amount')
    test['tip_amount'] = test['tip_amount'].fillna(test['tip_amount'].mean())
    print('done')      
    print('updating payment type')
    df = test['payment_type']
    df = pd.get_dummies(df)      
    test = test.drop('payment_type',axis=1)
    test = pd.concat([test,df],axis=1)
    print('done')      
    print('updating surcharge')      
    test['surcharge'] = test['surcharge'].fillna(test['surcharge'].mean())    
    print('done')
    print('creating distance....')      
    test['distance'] = test.apply(lambda row:distance(row['pickup_latitude'],row['pickup_longitude'],row['dropoff_latitude'],row['dropoff_longitude']),axis=1)
    me = test['distance'].mean()
    test['distance'] = test['distance'].apply(lambda x:update_distance(x,me))
    print('done')
    print('removing store and fwd flag field')
    test = test.drop('store_and_fwd_flag',axis=1)
    print('process complete')
    return test

In [11]:
def update_distance(x,me):
    if x==0:
        return me
    else:
        return x

In [12]:
train['vendor_id'] = train['vendor_id'].apply(lambda x:convert_vendorid(x))

In [13]:
train['new_user'] = train['new_user'].fillna('NO')

In [14]:
train['new_user'].isnull().sum()

0

In [15]:
train['new_user'] = pd.Series(np.where(train.new_user.values=='yes',1,0),train.index)

In [16]:
train['tip_amount'] = train['tip_amount'].fillna(train['tip_amount'].mean())

In [17]:
train['tip_amount'].isnull().sum()

0

In [18]:
df = train['payment_type']

In [19]:
df = pd.get_dummies(df)

In [20]:
train = train.drop('payment_type',axis=1)

In [21]:
train = pd.concat([train,df],axis=1)

In [22]:
train['surcharge'] = train['surcharge'].fillna(train['surcharge'].mean())

In [23]:
train.columns

Index(['TID', 'vendor_id', 'new_user', 'tolls_amount', 'tip_amount', 'mta_tax',
       'pickup_datetime', 'dropoff_datetime', 'passenger_count',
       'pickup_longitude', 'pickup_latitude', 'rate_code',
       'store_and_fwd_flag', 'dropoff_longitude', 'dropoff_latitude',
       'surcharge', 'fare_amount', 'CRD', 'CSH', 'DIS', 'NOC', 'UNK'],
      dtype='object')

In [24]:
train = convert_datetime(train)

In [25]:
train['total_jrny'] = cal(train)

In [26]:
train[train['total_jrny']<0]

,TID,vendor_id,new_user,tolls_amount,tip_amount,mta_tax,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,...,dropoff_longitude,dropoff_latitude,surcharge,fare_amount,CRD,CSH,DIS,NOC,UNK,total_jrny


In [27]:
train['distance'] = train.apply(lambda row:distance(row['pickup_latitude'],row['pickup_longitude'],row['dropoff_latitude'],row['dropoff_longitude']),axis=1)

In [28]:
train.isnull().sum()

TID                        0
vendor_id                  0
new_user                   0
tolls_amount               0
tip_amount                 0
mta_tax                    0
pickup_datetime            0
dropoff_datetime           0
passenger_count            0
pickup_longitude       49809
pickup_latitude        33101
rate_code                  0
store_and_fwd_flag    648122
dropoff_longitude       4987
dropoff_latitude        8162
surcharge                  0
fare_amount                0
CRD                        0
CSH                        0
DIS                        0
NOC                        0
UNK                        0
total_jrny                 0
distance                   0
dtype: int64

In [29]:
me = train['distance'].mean()

In [31]:
train['distance'] = train['distance'].apply(lambda x:update_distance(x,me))

In [32]:
train = train.drop('store_and_fwd_flag',axis=1)

In [33]:
train.isnull().sum()

TID                      0
vendor_id                0
new_user                 0
tolls_amount             0
tip_amount               0
mta_tax                  0
pickup_datetime          0
dropoff_datetime         0
passenger_count          0
pickup_longitude     49809
pickup_latitude      33101
rate_code                0
dropoff_longitude     4987
dropoff_latitude      8162
surcharge                0
fare_amount              0
CRD                      0
CSH                      0
DIS                      0
NOC                      0
UNK                      0
total_jrny               0
distance                 0
dtype: int64

In [34]:
train.to_csv('updated_train.csv',index=False)

In [37]:
test = pd.read_csv('test.csv')

In [38]:
test = update_test(test)

updating datetime
done
updating vendorid
done
updating new_user
done
updating tip amount
done
updating payment type
done
updating surcharge
done
creating distance....
done
removing store and fwd flag field
process complete


In [86]:
test.to_csv('updated_test.csv',index=False)

In [87]:
xgbmodel = XGBRegressor()

In [88]:
features = ['vendor_id','tolls_amount', 'tip_amount', 'mta_tax','passenger_count','rate_code','surcharge',
            'total_jrny', 'CRD','CSH', 'DIS', 'NOC', 'UNK', 'distance']

In [89]:
X = train[features]

In [90]:
y = np.sqrt(train['fare_amount'])

In [91]:
xgbmodel.fit(X,y)

XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=1, gamma=0,
       learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=100, nthread=-1,
       objective='reg:linear', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)

In [92]:
arr = xgbmodel.feature_importances_

In [93]:
for x,y in zip(features,arr):
    print(x,y)

vendor_id 0.0218023
tolls_amount 0.106105
tip_amount 0.175872
mta_tax 0.0174419
passenger_count 0.00290698
rate_code 0.155523
surcharge 0.0392442
total_jrny 0.209302
CRD 0.0348837
CSH 0.00436047
DIS 0.00872093
NOC 0.0116279
UNK 0.00290698
distance 0.209302


In [94]:
X_test = test[features]

In [95]:
pred = xgbmodel.predict(X_test)

In [96]:
pred = np.square(pred)

In [100]:
submission = pd.DataFrame(data=[], columns=['TID', 'fare_amount'])
submission.TID = test.TID; submission.fare_amount = pred
submission.to_csv('xgboost.csv', index=False)
submission.head()

,TID,fare_amount
0,AJX000119871,5.416895
1,AJX000119872,8.178459
2,AJX000119873,11.258808
3,AJX000119874,20.414660
4,AJX000119875,15.797602
